Sentiment analaysis is the determination of the feelings/emotions behind a text/sentence

In [1]:
!pip install vaderSentiment

In [2]:
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import transformers
from transformers import pipeline

In [3]:
df=pd.read_csv("book_reviews_sample.csv")

In [4]:
df.head()

,index,reviewText,rating
0,11494,Clean and funny. A bit busy with all the diffe...,3
1,984,Alex a sexy hot cop and the PhD candidate. Wha...,4
2,1463,Good thing that this is a free story. I read i...,1
3,10342,"Action, action, action! Equipment keeps gettin...",4
4,5256,this was hands down the worse book i have ever...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       100 non-null    int64 
 1   reviewText  100 non-null    object
 2   rating      100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


In [6]:
df['reviewText'][0]

'Clean and funny. A bit busy with all the different plots going on. But overall a good read. Bye now.'

In [7]:
df['reviewTextCleaned'] = df.apply(lambda x: re.sub(r"([^\w\s])","", x['reviewText']), axis=1)

In [8]:
df.head()

,index,reviewText,rating,reviewTextCleaned
0,11494,Clean and funny. A bit busy with all the diffe...,3,Clean and funny A bit busy with all the differ...
1,984,Alex a sexy hot cop and the PhD candidate. Wha...,4,Alex a sexy hot cop and the PhD candidate What...
2,1463,Good thing that this is a free story. I read i...,1,Good thing that this is a free story I read it...
3,10342,"Action, action, action! Equipment keeps gettin...",4,Action action action Equipment keeps getting b...
4,5256,this was hands down the worse book i have ever...,1,this was hands down the worse book i have ever...


In [9]:
vader_sentiment = SentimentIntensityAnalyzer()

In [10]:
df['vader_sentiment_score'] = df['reviewTextCleaned'].apply(lambda review: vader_sentiment.polarity_scores(review)['compound'])

In [11]:
df.head()

,index,reviewText,rating,reviewTextCleaned,vader_sentiment_score
0,11494,Clean and funny. A bit busy with all the diffe...,3,Clean and funny A bit busy with all the differ...,0.7684
1,984,Alex a sexy hot cop and the PhD candidate. Wha...,4,Alex a sexy hot cop and the PhD candidate What...,0.9325
2,1463,Good thing that this is a free story. I read i...,1,Good thing that this is a free story I read it...,0.6740
3,10342,"Action, action, action! Equipment keeps gettin...",4,Action action action Equipment keeps getting b...,0.6652
4,5256,this was hands down the worse book i have ever...,1,this was hands down the worse book i have ever...,-0.4767


In [12]:
#To classify the scores as positive, negative, neutral
bins = [-1, -0.1, 0.1, 1]
names = ['negative', 'neutral', 'positive']

df['vader_sentiment_label']=pd.cut(df['vader_sentiment_score'], bins, labels=names)

In [13]:
# df['vader_sentiment_label'].value_counts().plot.bar()

In [14]:
print("Hi")

Hi


In [15]:
transformer_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [16]:
transformer_labels = []
for review in df['reviewTextCleaned'].values:
    sentiment_list=transformer_pipeline(review)
    sentiment_label=[sent['label'] for sent in sentiment_list]
    transformer_labels.append(sentiment_label)


In [17]:
df['transformer_sentiment_label'] = transformer_labels

In [18]:
df.head()

,index,reviewText,rating,reviewTextCleaned,vader_sentiment_score,vader_sentiment_label,transformer_sentiment_label
0,11494,Clean and funny. A bit busy with all the diffe...,3,Clean and funny A bit busy with all the differ...,0.7684,positive,[POSITIVE]
1,984,Alex a sexy hot cop and the PhD candidate. Wha...,4,Alex a sexy hot cop and the PhD candidate What...,0.9325,positive,[POSITIVE]
2,1463,Good thing that this is a free story. I read i...,1,Good thing that this is a free story I read it...,0.6740,positive,[NEGATIVE]
3,10342,"Action, action, action! Equipment keeps gettin...",4,Action action action Equipment keeps getting b...,0.6652,positive,[NEGATIVE]
4,5256,this was hands down the worse book i have ever...,1,this was hands down the worse book i have ever...,-0.4767,negative,[NEGATIVE]


In [19]:
# df['transformer_sentiment_label'].value_counts().plot.bar()